In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn
import statsmodels.api as sm
import scipy as sc

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [177]:
df = pd.read_csv('train.csv')
df = df.set_index('id')
df.timestamp = pd.to_datetime(df.timestamp)
df.lat = df.lat.round(3)
#df.lat = ''.format()
#df = df.fillna(df.mean())

In [178]:
df.lon = df.lon.round(3)

df.head()

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,lat,lon,price_doc
id,,,,,,,,,,,,,,,,,,,,,
1,2011-08-20,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,13,22,1,0,52,4,55.89,37.60,5850000
2,2011-08-23,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,15,29,1,10,66,14,55.68,37.67,6000000
3,2011-08-27,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,11,27,0,4,67,10,55.70,37.74,5700000
4,2011-09-01,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1,4,4,0,0,26,3,55.84,37.35,13100000
5,2011-09-05,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,17,135,236,2,91,195,14,55.76,37.66,16331452


In [59]:
df_temp = pd.get_dummies(df)
df_temp.head()

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,sub_area_Vyhino-Zhulebino,sub_area_Zamoskvorech'e,sub_area_Zapadnoe Degunino,sub_area_Zjablikovo,sub_area_Zjuzino,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
id,,,,,,,,,,,,,,,,,,,,,
1,2011-08-20,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
2,2011-08-23,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
3,2011-08-27,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
4,2011-09-01,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
5,2011-09-05,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0


In [5]:
cov = df_temp.corr()
cov_temp = cov[cov!=1].fillna(0)

In [6]:
temp_min ={w:(cov_temp[w].argmin(),cov_temp[w].min()) for w in cov_temp.index}
temp_max ={w:(cov_temp[w].argmax(),cov_temp[w].max()) for w in cov_temp.index}

In [7]:
temp_min['price_doc'],temp_max['price_doc']

(('zd_vokzaly_avto_km', -0.28406855884521937),
 ('num_room', 0.47633653800603459))

In [8]:
df_temp = df.dropna()
df_grouped = df.groupby(['sub_area'])
floor = df_temp.floor
price = df_temp.price_doc

In [10]:

'''
for name,group in df_grouped:
    plt.figure()
    plt.title(name)
    plt.scatter(group.max_floor,group.price_doc,label=name)

plt.legend()    
plt.show()
'''

'\nfor name,group in df_grouped:\n    plt.figure()\n    plt.title(name)\n    plt.scatter(group.max_floor,group.price_doc,label=name)\n\nplt.legend()    \nplt.show()\n'

In [9]:
from collections import *
dict_building = defaultdict()
l = lambda x:dict_building[x].append(df.price_doc)

In [10]:
df_temp['building'] = df_temp.build_year.map(int).map(str)+ df_temp.sub_area + df_temp.material.map(int).map(str)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_grouped1 = df_temp.groupby('building')['price_doc'].count()
df_grouped = df_temp.groupby('building')['price_doc'].mean()

In [55]:
##Mean price grouped by build year, area and material if the count is greater than 10. 

#df_grouped[df_grouped1[df_grouped1>10].index]

In [13]:
col_train=['floor','full_sq','life_sq','max_floor','num_room','price_doc','area_m','material']
col_maxfloor_train = ['floor','full_sq','life_sq','price_doc','area_m']
df_train = df.loc[:,col_train].dropna()
df_maxfloor = df_train.loc[:,col_maxfloor_train]

In [119]:
from sklearn.model_selection import train_test_split
X_train_maxfloor, X_test_maxfloor, y_train_maxfloor, y_test_maxfloor = train_test_split(df_maxfloor.values,df_train.max_floor.values)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_maxfloor)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_maxfloor = scaler.transform(X_train_maxfloor)
X_test_maxfloor = scaler.transform(X_test_maxfloor)

In [134]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes = (30,),solver='sgd',learning_rate_init=0.01,max_iter=500,activation='tanh')

mlp.fit(X_train_maxfloor, y_train_maxfloor)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30,), learning_rate='constant',
       learning_rate_init=0.01, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [135]:
float(np.sum(mlp.predict(X_test_maxfloor) == y_test_maxfloor))/float(len(X_test_maxfloor))

0.38663484486873506

In [ ]:
df_maxfloor_pred = df[pd.isnull(df.max_floor)].loc[:,col_maxfloor_train].dropna()

In [ ]:
pred_maxfloor = mlp.predict(X_test_maxfloor)

In [37]:
cols=['state','price_doc']
state = df[~pd.isnull(df.state)*df.state!=33].loc[:,cols]

In [43]:
state_grouped = state.groupby('state').describe()

In [46]:
#state.groupby('state').describe()

In [179]:
#####Missing Value Functions

In [180]:
def fill_material(df,method='loc'):
    if method == 'year':
        df_mat = df[~pd.isnull(df.material) * ~pd.isnull(df.build_year)]
        df_mat_grouped = df_mat.groupby(['build_year'])
        mat={}
        for name,group in df_mat_grouped:
            grp = group.groupby('material').count()['price_doc']
            mat[name] = np.argmax(grp)
        df_mat_null = df[pd.isnull(df.material)].set_index('build_year')
        df_mat_null.material = df_mat_null.material.map(mat)
        df_mat_null = df_mat_null.reset_index()
        df_mat_notnull = df[~pd.isnull(df.material)]
        df = pd.concat([df_mat_notnull,df_mat_null])
        #df = df.set_index('id')
    if method =='loc':
        df['latlon'] = [(x,y) for x,y in zip(df.lat,df.lon)]
        df_mat = df[~pd.isnull(df.lat) * ~pd.isnull(df.lon) * ~pd.isnull(df.material)]
        df_mat_grouped = df_mat.groupby(['latlon'])
        mat={}
        for name,group in df_mat_grouped:
            grp = group.groupby('material').count()['price_doc']
            mat[(round(name[0],3),round(name[1],3))] = np.argmax(grp)
        df_mat_null = df[pd.isnull(df.material)]
        df_mat_null.material = df_mat_null.latlon.map(mat).values
        df_mat_notnull = df[~pd.isnull(df.material)]
        df = pd.concat([df_mat_notnull,df_mat_null])
        #df = df.remove('latlon')
    return df


In [181]:
def fill_life_sq(df):
    df_life_notnull = df[~pd.isnull(df.life_sq) & ~pd.isnull(df.full_sq)]
    df_life_notnull['ratio'] = df_life_notnull.life_sq/df_life_notnull.full_sq
    mean = df_life_notnull.ratio.mean()
    df_life_null = df[pd.isnull(df.life_sq)]
    df_life_null.life_sq = df_life_null.full_sq * mean
    df_life_notnull = df[~pd.isnull(df.life_sq)]
    df = pd.concat([df_life_notnull,df_life_null])
    return df

In [182]:
def fill_maxfloor(df):
    df['latlon'] = [(x,y) for x,y in zip(df.lat,df.lon)]
    df_maxfloor = df[~pd.isnull(df.lat) & ~pd.isnull(df.lon) & ~pd.isnull(df.max_floor)]
    df_maxfloor_grouped = df_maxfloor.groupby(['latlon'])
    maxf={}
    for name,group in df_maxfloor_grouped:
        grp = group.groupby('max_floor').count()['price_doc']
        maxf[(round(name[0],3),round(name[1],3))] = np.argmax(grp)
    df_maxf_null = df[pd.isnull(df.max_floor)]
    df_maxf_null.max_floor = df_maxf_null.latlon.map(maxf)
    df_maxf_notnull = df[~pd.isnull(df.max_floor)]
    df = pd.concat([df_maxf_notnull,df_maxf_null])
    #df = df.drop('latlon')
    return df

In [192]:
def fill_buildyear(df,method='loc'):
    if method =='material':
        df_year = df[~pd.isnull(df.material) & ~pd.isnull(df.build_year)]
        df_year_grouped = df_year.groupby('material')
        maxy={}
        for name,group in df_year_grouped:
            grp = group.groupby('build_year').count()['price_doc']
            maxy[name] = np.argmax(grp)
        df_year_null = df[pd.isnull(df.build_year)]
        df_year_null.build_year = df_year_null.material.map(maxy)
        df_year_notnull = df[~pd.isnull(df.build_year)]
        df = pd.concat([df_year_notnull,df_year_null])
        #df = df.drop('latlon')
    elif method =='loc':
        df['latlon'] = [(x,y) for x,y in zip(df.lat,df.lon)]
        df_year = df[~pd.isnull(df.lat) & ~pd.isnull(df.lon) & ~pd.isnull(df.build_year)]
        df_year_grouped = df_year.groupby(['latlon'])
        maxy={}
        for name,group in df_year_grouped:
            grp = group.groupby('build_year').count()['price_doc']
            maxy[(round(name[0],3),round(name[1],3))] = np.argmax(grp)
        df_year_null = df[pd.isnull(df.build_year)]
        df_year_null.build_year = df_year_null.latlon.map(maxy)
        df_year_notnull = df[~pd.isnull(df.build_year)]
        df = pd.concat([df_year_null,df_year_notnull])
    #df = df.drop('latlon')   
    
    return df  


    

In [330]:
def fill_state(df):
    df_year = df[~pd.isnull(df.state) & ~pd.isnull(df.build_year)]
    df_year_grouped = df_year.groupby('build_year')
    maxy={}
    for name,group in df_year_grouped:
        grp = group.groupby('state').count()['price_doc']
        maxy[name] = np.argmax(grp)
    df_year_null = df[pd.isnull(df.state)]
    df_year_null.build_year = df_year_null.build_year.map(maxy)
    df_year_notnull = df[~pd.isnull(df.state)]
    df = pd.concat([df_year_notnull,df_year_null])
    #df = df.drop('latlon')
    return df

In [193]:
temp = fill_material(df,'loc')

In [194]:
temp = fill_buildyear(temp,'loc')

In [195]:
temp = fill_maxfloor(temp)

In [331]:
df_macro = pd.read_csv('macro 2.csv')
df_macro.timestamp = pd.to_datetime(df_macro['timestamp'])

In [332]:
df_macro.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2010-01-01,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,2010-01-02,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,2010-01-03,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,2010-01-04,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.905,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,2010-01-05,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.836,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


In [333]:
df_temp = pd.merge(df,df_macro,left_on='timestamp',right_on='timestamp')

In [334]:
df_macro.timestamp = pd.to_datetime(df_macro['timestamp'])

In [335]:
def add_yearmonth(df_macro):
    df_macro['year'] = df_macro['timestamp'].map(lambda x: x.year)
    df_macro['month'] = df_macro['timestamp'].map(lambda x: x.month)
    df_macro['ym'] = df_macro['year'].map(str) + df_macro['month'].map(str)
    return df_macro

In [336]:
df = add_yearmonth(df)

In [337]:
df_macro = add_yearmonth(df_macro)

In [338]:
df_macro_month = df_macro.groupby('ym').mean()

In [339]:
df_macro_month = df_macro_month.iloc[:,:-2]

In [340]:
df_macro_month = df_macro_month.reset_index()
df_macro_month.head()

,ym,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,20101,76.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.838164,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,201010,81.49,12086.5,3.8,329.3,368.9,NaN,11.883,2.2,30.327826,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,201011,84.90,12086.5,3.8,330.9,377.1,NaN,12.029,2.2,31.005903,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,201012,89.56,12086.5,3.8,333.6,393.7,NaN,11.810,2.2,30.806294,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,20102,72.97,NaN,NaN,315.1,337.2,NaN,18.114,NaN,30.157571,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


In [341]:
df_data_wmacro = pd.merge(df,df_macro_month,left_on='ym',right_on='ym')

In [343]:
df_train = fill_material(df_data_wmacro)

In [344]:
df_train = fill_maxfloor(df_train)

In [345]:
df_train = fill_buildyear(df_train)

In [346]:
df_train = fill_life_sq(df_train)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [348]:
df_train = fill_state(df_train)

In [349]:
df_test = pd.read_csv('test.csv',parse_dates=['timestamp'])


In [350]:
# ylog will be log(1+y), as suggested by https://github.com/dmlc/xgboost/issues/446#issuecomment-135555130
ylog_train_all = np.log1p(df_train['price_doc'].values)
id_test = df_test['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

# Build df_all = (df_train+df_test).join(df_macro)
num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
#df_all = pd.merge_ordered(df_all, df_macro, on='timestamp', how='left')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp'], axis=1, inplace=True)

(38132, 392)


In [351]:
# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj: 
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)

In [352]:
# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

# Create a validation set, with last 20% of data
num_val = int(num_train * 0.2)

X_train_all = X_all[:num_train]
X_train = X_all[:num_train-num_val]
X_val = X_all[num_train-num_val:num_train]
ylog_train = ylog_train_all[:-num_val]
ylog_val = ylog_train_all[-num_val:]

X_test = X_all[num_train:]

df_columns = df_values.columns

print('X_train_all shape is', X_train_all.shape)
print('X_train shape is', X_train.shape)
print('y_train shape is', ylog_train.shape)
print('X_val shape is', X_val.shape)
print('y_val shape is', ylog_val.shape)
print('X_test shape is', X_test.shape)

(38132, 396)
('X_train_all shape is', (30470, 396))
('X_train shape is', (24376, 396))
('y_train shape is', (24376,))
('X_val shape is', (6094, 396))
('y_val shape is', (6094,))
('X_test shape is', (7662, 396))


In [353]:
X_train_all[np.isnan(X_train_all)]=-99999
ylog_train_all[np.isnan(ylog_train_all)]=-99999
X_train[np.isnan(X_train)]=-99999
ylog_train[np.isnan(ylog_train)]=-99999
X_val[np.isnan(X_val)]=-99999
ylog_val[np.isnan(ylog_val)]=-99999
X_test[np.isnan(X_test)]=-99999

In [354]:
dtrain_all = xgb.DMatrix(X_train_all, ylog_train_all,missing=-99999)
dtrain = xgb.DMatrix(X_train, ylog_train,missing=-99999)
dval = xgb.DMatrix(X_val, ylog_val,missing=-99999)
dtest = xgb.DMatrix(X_test,missing=-99999)

In [355]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

# Uncomment to tune XGB `num_boost_rounds`
partial_model = xgb.train(xgb_params, dtrain, num_boost_round=1000, evals=[(dval, 'val')],
                       early_stopping_rounds=50, verbose_eval=20)

num_boost_round = partial_model.best_iteration

[0]	val-rmse:14.3033
Will train until val-rmse hasn't improved in 50 rounds.
[20]	val-rmse:5.13345
[40]	val-rmse:1.89617
[60]	val-rmse:0.804532
[80]	val-rmse:0.501176
[100]	val-rmse:0.437554
[120]	val-rmse:0.425477
[140]	val-rmse:0.421823
[160]	val-rmse:0.420396
[180]	val-rmse:0.419696
[200]	val-rmse:0.419493
[220]	val-rmse:0.419198
[240]	val-rmse:0.418781
[260]	val-rmse:0.418949
[280]	val-rmse:0.418942
Stopping. Best iteration:
[241]	val-rmse:0.41874



In [356]:
model = xgb.train(dict(xgb_params, silent=0), dtrain_all, num_boost_round=num_boost_round)

In [357]:
ylog_pred = model.predict(dtest)
y_pred = np.exp(ylog_pred) - 1

df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

In [358]:
df_sub.to_csv('submission.csv', index=False)